# 线性神经网络

### 线性回归
数学模型:
$$
\hat y = \mathbf w^T\mathbf x
$$
损失函数：
$$
L(\mathbf w,b) = \frac 1 n\sum_{i=1}^n \frac 1 2 \big(\mathbf w^T\mathbf x +b-y^{(i)}\big)^2
$$
目标：
$$
\mathbf w^{best},b^{best} = \operatorname*\argmin _{\mathbf w,b} L(\mathbf w,b)
$$

In [9]:
%matplotlib inline
import torch
import math
import numpy as np
import time

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'cuda'

In [10]:
class Timer:  #@save
    """记录多次运行时间"""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """返回时间总和"""
        return sum(self.times)

    def cumsum(self):
        """返回累计时间"""
        return torch.cumsum(torch.tensor(self.times),-1).tolist()

In [32]:
n = 10000
a = torch.full((n,),23,dtype=torch.float32,device=device)
b = torch.linspace(0,14313,n,dtype=torch.float32,device=device)
# 循环计算
c = torch.zeros(n,dtype=torch.float32,device=device)
timer = Timer()
timer.start()
for i in range(n):
    c[i] = a[i] + b[i]
s1 = timer.stop()

# 矢量计算
timer.start()
d = a+b
s2 = timer.stop()
f'循环计算耗时: {s1:.5f}，矢量计算耗时: {s2:.5f} sec, 且计算结果和循环对比:{d.equal(c)}'


'循环计算耗时: 0.10243，矢量计算耗时: 0.00004 sec, 且计算结果和循环对比:True'

一条重要结论：在高斯噪声的假设下，最小化均方误差等价于对线性模型的极大似然估计。证明过程：  
 由于假设显性模型的输出中增加了高斯噪声
 $$
 y=\mathbf w^T\mathbf x + b + \epsilon, 其中 \epsilon \sim \mathcal N(0,\sigma^2)
 $$
 
 因此可以写出给定x观测到y的似然：
 
 $$
 P(y|\mathbf x) = \frac 1 {\sqrt {2\pi}\sigma}exp\Big(-\frac {1} {2\sigma^2} (y - \mathbf w^T\mathbf x - b) \Big)
 $$

 然后根据极大似然估计: **模型参数的最优值是使整个数据集的似然最大的值**。整个数据集都发生的似然为：

 $$
 p(\mathbf y | \mathbf X) = \prod_{i=1}^{n}p(y^{(i)}|\mathbf x^{(i)})
 $$

 取负对数：
 
 $$
 -\log  p(\mathbf y | \mathbf X) = \sum_{i=1}^n \frac 1 2 \log (2\pi \sigma^2) + \frac 1 {2\sigma^2}\Big( y^{(i)} - \mathbf w^T\mathbf x^{(i)} - b\Big)^2
 $$

 最后一个式子即等价于均方误差MSE
 

In [33]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)